In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from Supply import Supply

In [2]:
model = Supply()
folder = "/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Replicating the Innovators Dilemma/1 Summary Statistics/Data"
model.setup(folder)

In [3]:
results = model.MC()

dPdQ = results['dPdQ']
ePQ = results['ePQ']
ABCDEFGH = results['ABCDEFGH']
q = results['q']
MC = results['MC']

In [5]:
print(q[:, 1])

q_copy = q.copy()
q_copy[:, 1] = q[:, 0]
q_copy[0, 3] = q[1, 3]
q_copy[:, 2] = q_copy[:, 3]


[   0.            0.          102.10909091  224.37272727  279.19090909
  486.58        728.5         811.96        844.91111111  873.58571429
  407.2         258.15        159.18         76.34         83.06
  913.         1114.56       1017.5       ]


In [11]:
par = model.par
No = par.State[13,0]
Nb = par.State[13,1]
Nn = par.State[13,2]

model.foc(q,MC,13, No, Nb, Nn)

array([27.81259037])

In [ ]:
par = model.par
print(par.State[13,0])
print(par.State[13,1])
print(par.State[13,2])

In [ ]:
print(MC[13,0])
print(MC[13,1])
print(MC[13,2])
print(MC[13,3])

In [12]:
alpha1 = -3.28454
alpha2 = 0.909773
alpha3 = 1.204684
Mkt = 56638.8
X_o = 6.490788
X_n = 6.240812
Xe_o = -0.4157396
Xe_n = 0.4135973
YearDummy = -5.653561
MC_o = 1.1213105727517398
MC_n = -2.916521500397822
No = 1
Nb = 4
Nn = 5

def foc(q):
    Qo = No * q[13,0] + Nb * q[13,1]
    Qn = Nb * q[13,2] + Nn * q[13,3]
    Q0 = Mkt - Qo - Qn

    Po = (-1 / alpha1) * (-np.log(Qo / Q0) + alpha2 * 0 + alpha3 * X_o + Xe_o + YearDummy)
    Pn = (-1 / alpha1) * (-np.log(Qn / Q0) + alpha2 * 1 + alpha3 * X_n + Xe_n + YearDummy)

    dPoQo = (Qo + Q0) / (alpha1 * Qo * Q0)
    dPnQo = 1 / (alpha1 * Q0)
    dPoQn = 1 / (alpha1 * Q0)
    dPnQn = (Qn + Q0) / (alpha1 * Qn * Q0)

    foc_o  = Po + dPoQo * q[13,0] - MC_o
    foc_bo = Po + dPoQo * q[13,1] + dPnQo * q[13,2] - MC_o
    foc_bn = Pn + dPnQn * q[13,2] + dPoQn * q[13,1] - MC_n
    foc_n  = Pn + dPnQn * q[13,3] - MC_n  

    F = foc_o**2 + foc_bo**2 + foc_bn**2 + foc_n**2

    return F

foc(q)

27.812590370735805